# 🏎️ Taller POO: Sistema de Gestión "Alta Gama Motors"

**Rol:** Ingeniero de Software Senior.
**Cliente:** Concesionaria de lujo "Alta Gama Motors".
**Misión:** El dueño está furioso. El sistema anterior permitía errores graves: autos con kilometraje negativo y ventas de vehículos sin precio. Tu tarea es reconstruir el núcleo del sistema aplicando **Programación Orientada a Objetos** para garantizar la seguridad y la reutilización del código.

**Conceptos Clave del IB (Tema B3):**
*   **B3.1.5 Encapsulamiento:** Proteger la integridad de los datos (el kilometraje no se toca).
*   **B3.2.1 Herencia:** Reutilizar código entre autos estándar y eléctricos.
*   **Defensa del Código:** Uso de validaciones y manejo de errores.

## 1. La Clase Base: El Vehículo (Encapsulamiento)

Primero diseñaremos el plano general. Aplicaremos **Encapsulamiento** para proteger el atributo más sensible de un auto usado: su kilometraje. Nadie debería poder rebajarlo para estafar a un cliente.

**Regla de Negocio:** El kilometraje es privado. Solo puede aumentar, nunca disminuir.

In [ ]:
class Vehiculo:
    """
    Clase base que representa cualquier vehículo de la flota.
    """
    
    def __init__(self, marca: str, modelo: str, año: int) -> None:
        # Inicialización explícita (Estándar Técnico)
        self.marca: str = marca
        self.modelo: str = modelo
        self.año: int = año
        
        # ATRIBUTO PRIVADO (Encapsulamiento B3.1.5)
        # Usamos __ para que no sea accesible directamente desde fuera
        self.__kilometraje: float = 0.0

    # --- GETTER (Acceso de Lectura) ---
    def get_kilometraje(self) -> float:
        return self.__kilometraje

    # --- SETTER CON VALIDACIÓN (Acceso de Escritura Controlado) ---
    def actualizar_kilometraje(self, nuevos_km: float) -> None:
        """
        Actualiza el kilometraje solo si el valor es lógico.
        No permite retroceder el contador (evita fraudes).
        """
        try:
            # Validación de integridad
            if nuevos_km >= self.__kilometraje:
                self.__kilometraje = nuevos_km
                print(f"✅ Kilometraje actualizado a {self.__kilometraje} km.")
            else:
                # Levantamos un error intencional si intentan hacer trampa
                raise ValueError(f"Intento de fraude: No se puede bajar de {self.__kilometraje} a {nuevos_km}.")
        
        except ValueError as e:
            print(f"⛔ ALERTA DE SEGURIDAD: {e}")

    def describir(self) -> str:
        return f"{self.año} {self.marca} {self.modelo}"

# --- PRUEBA DE LA CLASE BASE ---
auto_1 = Vehiculo("Toyota", "Corolla", 2022)
print(f"Vehículo creado: {auto_1.describir()}")

# Intentamos actualizar legalmente
auto_1.actualizar_kilometraje(1500.5)

# Intentamos 'hackear' el kilometraje (debería fallar)
print("\n--- Intentando adulterar odómetro ---")
auto_1.actualizar_kilometraje(500) # Menor al actual

> **🧠 Pregunta para Pensar:** Si el atributo `__kilometraje` fuera público (sin los guiones bajos), ¿qué impediría que un programador escribiera `auto_1.kilometraje = -100`? ¿Por qué esto es un riesgo para la empresa?

## 2. Especialización: El Auto Eléctrico (Herencia)

La concesionaria ha empezado a vender autos Tesla. Estos autos *son* Vehículos (tienen marca, modelo, ruedas), pero tienen características únicas: batería y autonomía.

**Concepto IB B3.2.1:** Usaremos **Herencia** para no reescribir el código que ya hicimos. La clase hija (`AutoElectrico`) heredará todo de `Vehiculo` y agregará lo suyo.

In [ ]:
class AutoElectrico(Vehiculo):
    """
    Subclase que representa vehículos a batería.
    Hereda atributos y métodos de la clase Vehiculo.
    """
    
    def __init__(self, marca: str, modelo: str, año: int, capacidad_bateria: int) -> None:
        # 1. Inicializamos la parte 'Vehiculo' usando super()
        # Esto llama al __init__ de la clase padre
        super().__init__(marca, modelo, año)
        
        # 2. Inicializamos los atributos exclusivos del eléctrico
        self.capacidad_bateria: int = capacidad_bateria # en kWh
        self.nivel_carga: int = 100 # Porcentaje

    def cargar_bateria(self) -> None:
        """Simula la carga del vehículo."""
        print(f"🔌 Cargando {self.marca} {self.modelo}...")
        self.nivel_carga = 100
        print("⚡ Carga completa al 100%.")

    # Sobrescritura de método (Polimorfismo B3.2.2)
    # Modificamos 'describir' para que incluya datos de la batería
    def describir(self) -> str:
        descripcion_base: str = super().describir()
        return f"{descripcion_base} [ELÉCTRICO - Batería: {self.capacidad_bateria}kWh]"

# --- PRUEBA DE HERENCIA ---
tesla = AutoElectrico("Tesla", "Model 3", 2024, 75)

# Usamos un método propio de la clase hija
tesla.cargar_bateria()

# Usamos un método HEREDADO de la clase padre (¡Reutilización de código!)
print("\n--- Probando herencia del odómetro ---")
tesla.actualizar_kilometraje(120.5)

# Probamos el polimorfismo en la descripción
print(f"Info: {tesla.describir()}")

## 3. Gestión de la Flota (Arrays de Objetos)

Ahora necesitamos gestionar el inventario completo. Usaremos una lista (Array) para guardar diferentes objetos. Aquí veremos el poder del **Polimorfismo**: la lista puede contener mezclas de `Vehiculo` y `AutoElectrico` y tratarlos a todos por igual.

In [ ]:
class Concesionaria:
    def __init__(self, nombre: str) -> None:
        self.nombre: str = nombre
        # Lista que almacenará objetos de tipo Vehiculo (o sus hijos)
        self.inventario: list[Vehiculo] = []

    def agregar_auto(self, auto: Vehiculo) -> None:
        self.inventario.append(auto)
        print(f"📥 Ingresó al stock: {auto.marca} {auto.modelo}")

    def mostrar_inventario(self) -> None:
        print(f"\n🏢 Inventario de {self.nombre}:")
        print("-" * 40)
        
        if not self.inventario:
            print("El inventario está vacío.")
            return

        # Recorremos la lista de objetos
        for auto in self.inventario:
            # Polimorfismo: llama al método describir() correspondiente 
            # (sea el del padre o el del hijo) automáticamente.
            print(f"🚗 {auto.describir()} | KM: {auto.get_kilometraje()}")

# --- EJECUCIÓN PRINCIPAL ---
agencia = Concesionaria("Alta Gama Motors")

# Creamos instancias
v1 = Vehiculo("Ford", "Mustang", 1969)
v2 = AutoElectrico("Porsche", "Taycan", 2023, 93)
v3 = Vehiculo("Fiat", "600", 1980)

# Modificamos estados
v1.actualizar_kilometraje(85000)
v2.actualizar_kilometraje(5000)

# Agregamos a la flota
agencia.agregar_auto(v1)
agencia.agregar_auto(v2)
agencia.agregar_auto(v3)

# Reporte final
agencia.mostrar_inventario()

### 🏁 Ticket de Salida (Metacognición)

1.  **Seguridad:** En el método `actualizar_kilometraje`, usamos un bloque `try...except`. ¿Por qué es mejor lanzar un error (`raise ValueError`) que simplemente no hacer nada y quedarse callado cuando alguien intenta bajar el kilometraje?
2.  **Herencia:** Si Alta Gama Motors decide vender **Motos**, ¿de qué clase debería heredar la nueva clase `Moto`? ¿De `Vehiculo` o de `AutoElectrico`? Justifica tu respuesta pensando en los atributos que comparten.